In [ ]:
import msmrd2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Define Patchy Particle potential
sigma = 1.0
strength = 100.0
patchesCoordinates = [np.array([1.,0.,0.])]
patchyParticle = msmrd2.potentials.patchyParticle(sigma, strength, patchesCoordinates)

In [ ]:
dist = np.arange(0, 2, 0.01)
pos1 = np.array([0., 0., 0.])
pos2 = np.array([[i, 0., 0.] for i in dist])
phi1 = 0.0
phi2 = np.pi
phi1p = np.pi/4.0
phi2p = 0.0
u = np.array([0.,0.,1.]) #rotationAxis
# Quaternions that rotate phi along axis u
theta1 = np.array([np.cos(0.5*phi1 ),
                   np.sin(0.5*phi1 )*u[0],
                   np.sin(0.5*phi1 )*u[1],
                   np.sin(0.5*phi1 )*u[2]]) 
theta2 = np.array([np.cos(0.5*phi2 ),
                   np.sin(0.5*phi2 )*u[0],
                   np.sin(0.5*phi2 )*u[1],
                   np.sin(0.5*phi2 )*u[2]]) 
theta1p = np.array([np.cos(0.5*phi1p ),
                   np.sin(0.5*phi1p )*u[0],
                   np.sin(0.5*phi1p )*u[1],
                   np.sin(0.5*phi1p )*u[2]]) 
theta2p = np.array([np.cos(0.5*phi2p ),
                   np.sin(0.5*phi2p )*u[0],
                   np.sin(0.5*phi2p )*u[1],
                   np.sin(0.5*phi2p )*u[2]]) 

potantialAligned = [patchyParticle.evaluate(pos1,p2,theta1,theta2) for p2 in pos2]
potantialMisaligned = [patchyParticle.evaluate(pos1,p2,theta1p,theta2p) for p2 in pos2]

In [ ]:
plt.plot(dist, potantialAligned, label = "aligned")
plt.plot(dist, potantialMisaligned, label = "misaligned")
plt.plot(dist, 0*dist, '--k', lw=0.5)
plt.legend()
plt.xlim([0,2])

In [ ]:
patchyParticle.forceTorque(pos1,pos2[20],theta1p,theta2p)

In [ ]:
def U(x, sigma, xstar, xcritical, eps, a, b):
    if (xstar>xcritical):
        raise ValueError('xstar should be smaller than xcritical.')
    resultm2 = np.array([])
    resultm1 = np.array([])
    result0 = np.array([])
    x0 = x[x<0]
    if len(x0) > 0:
        xm2 = x0[x0 < -xcritical]
        resultm2 = 0.0*xm2
        xm1 = x0[x0 >= -xcritical]
        xm1 = xm1[xm1 < -xstar]
        resultm1 = eps*b*(xcritical/sigma + xm1/sigma)**2
        x0 = x0[x0 >= -xstar]
        result0 = eps*(1.0 - a*(x0/sigma)**2)
    x1 = x[x>=0]
    x1 = x1[x1<xstar]
    result1 = eps*(1.0 - a*(x1/sigma)**2)
    x2 = x[x>=xstar]
    x2 = x2[x2<xcritical]
    result2 = eps*b*(xcritical/sigma - x2/sigma)**2
    x3 = x[x>=xcritical]
    result3 = 0.0*x3 
    return np.concatenate((resultm2, resultm1, result0, result1, result2, result3))

In [ ]:
def calculateContinuityParametrs(sigma, xstar, a):
    xcritical = sigma**2/(a*xstar)
    b = (1.0 - a*(xstar/sigma)**2)/(xcritical/sigma - xstar/sigma)**2
    return xcritical, b
def calculateContinuityParametrs2(sigma, xstar, a):
    xcritical = sigma**2/(a*xstar)
    b = -(1.0 - a*(xstar/sigma)**2)/(xcritical/sigma - xstar/sigma)**2
    return xcritical, b

In [ ]:
sigma = 1.0
# Repulsive part
eps = 100.0
a = 1.0
xstar = 0.85*sigma
xcritical, b = calculateContinuityParametrs(sigma,xstar,a)
# Attractive patches part
eps2 = -20.0
a2 = 20.0
xstar2 = 0.1*sigma
xcritical2, b2 = calculateContinuityParametrs(sigma,xstar2,a2)
# Isotropic attractive
eps3 = -10.0
a3 = 0.5#0.55
xstar3 = 0.85*sigma
xcritical3, b3 = calculateContinuityParametrs(sigma,xstar3,a3)

In [ ]:
x = np.arange(0,2.0*sigma,0.001)
y1 = U(x, sigma, xstar, xcritical, eps, a, b)
y21 = U(x - sigma, sigma, xstar2, xcritical2, eps2, a2, b2)
y22 = U(x + sigma, sigma, xstar2, xcritical2, eps2, a2, b2)
y3 = U(x, sigma, xstar3, xcritical3, eps3, a3, b3)
ym = y1 + y21 + y3
yp = y1 + y22 + y3
plt.plot(x,ym, '-b')
plt.plot(x,yp, '-r')
plt.plot(x, 0*x, '--k', lw=0.5)
plt.xlim([0, 2*sigma])
#plt.ylim([-25.0, 100])
